# Aprimorando o Ranqueamento de Documentos usando Grafos de Conhecimento

### Nicholas Valle Gross - 122101661

In [ ]:
!pip install spacy wikipedia-api SPARQLWrapper networkx
!python -m spacy download pt_core_news_lg

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 15.4 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=e78fa3447997584aafd70935dd4ad7a1c1cca9015e77b70afd6043a3776a3385
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#!pip install https://github.com/explosion/spacy-models/releases/download/pt_core_news_lg-3.7.0/pt_core_news_lg-3.7.0-py3-none-any.whl

In [ ]:
import spacy
import wikipediaapi
import math
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
import matplotlib.pyplot as plt
from functools import lru_cache
import os
import re
import json
import requests
from zipfile import ZipFile
from bs4 import BeautifulSoup
import time
import requests
import urllib.parse
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np

In [ ]:
nlp = spacy.load("pt_core_news_lg")
wiki = wikipediaapi.Wikipedia(user_agent='your-user-agent', language='pt')

In [ ]:
#imprime informações semânticas sobre os tokens do documento
def doc_info(doc):
    print("Text: ", doc.text)
    print("Named Entities: ", doc.ents)
    print("Tokens:")
    for token in doc:
        print("\t", token.text, token.pos_, token.dep_)

#verifica se um termo tem página na wikipedia
def existe_na_wikipedia(frase):
    page = wiki.page(frase)
    return page.exists()

def visualizar_grafo(G, titulo="Knowledge Graph"):
    pos = nx.spring_layout(G, k=0.5)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=4000, font_size=14)
    labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=12)
    plt.title(titulo)
    plt.show()

##  Extração de Entidades

In [ ]:
ARTIGOS = {"o","a","os","as","um","uma","uns","umas"}
QUANTIFICADORES = {
    "muitos","muitas","alguns","algumas","vários","várias",
    "todo","toda","todos","todas"
}

COPULAS = {"ser","estar","ficar","permanecer","tornar"}

In [ ]:
# ------------------------------------------------------
#            EXTRAÇÃO DE ENTIDADES COMPOSTAS
# ------------------------------------------------------

def extract_noun_chunk_compounds(doc):
    """Extrai compostos provenientes de noun_chunks."""
    compounds = set()
    for chunk in doc.noun_chunks:

        tokens = [t.text for t in chunk]

        # Remove artigo
        if tokens[0].lower() in ARTIGOS:
            tokens = tokens[1:]

        if len(tokens) <= 1:
            continue

        # Remove quantificadores
        if tokens[0].lower() in QUANTIFICADORES:
            continue

        compounds.add(" ".join(tokens))

    return compounds


def extract_amod_compounds(doc):
    """Extrai compostos NOUN + ADJ (ex.: ponto turístico)."""
    compounds = set()
    for token in doc:
        if token.pos_ == "NOUN":
            adjs = [c for c in token.children if c.dep_ == "amod" and c.pos_ == "ADJ"]
            if adjs:
                phrase = " ".join([token.text] + [a.text for a in adjs])
                compounds.add(phrase)
    return compounds

def extract_prepositional_compounds(doc):
    """Extrai compostos NOUN + ADP + NOUN."""
    compounds = set()
    for token in doc:
        if token.pos_ in ("NOUN","PROPN"):
            for child in token.children:
                if child.pos_ in ("NOUN","PROPN") and child.dep_ in ("nmod","obl","nmod:poss"):
                    preps = [w.text for w in child.children if w.dep_ == "case"]
                    if preps:
                        prep = preps[0]
                        compounds.add(f"{token.text} {prep} {child.text}")
    return compounds


def filter_compounds(chunk_comp, amod_comp, prep_comp):
    """
    Heurística final:
    - prioriza noun_chunks e amod
    - incluir preposicional somente se:
        (a) aparece como subparte de outro composto OU
        (b) existe na Wikipédia
    """
    final = set(chunk_comp) | set(amod_comp)

    for p in prep_comp:
        if any(p in c or c in p for c in final):
            final.add(p)
            continue
        if existe_na_wikipedia(p):
            final.add(p)

    return final

def span_text(token, token_map): return token_map.get(token.i, token.text)

def build_token_map(doc, compounds):
    """Mapeia token.i → texto do composto correspondente."""
    token_map = {}
    toks = [t.text for t in doc]

    for comp in compounds:
        parts = comp.split()
        L = len(parts)
        for i in range(len(toks)-L+1):
            if toks[i:i+L] == parts:
                for j in range(i, i+L):
                    token_map[j] = comp
    return token_map

def lemmatize_string(s, nlp):
    return " ".join([t.lemma_ for t in nlp(s)])

def normalize_entity(e):
    return e.strip().lower()

def extrair_entidades(doc):
    """Extrai entidades nomeadas, compostas e substantivos simples."""
    nomeadas = {ent.text for ent in doc.ents}

    chunk_comp = extract_noun_chunk_compounds(doc)
    amod_comp = extract_amod_compounds(doc)
    prep_comp = extract_prepositional_compounds(doc)

    compostas = filter_compounds(chunk_comp, amod_comp, prep_comp)

    comp_lem = set()
    for comp in compostas:
        comp_lem.add(lemmatize_string(comp, nlp))

    substantivos = set()
    for t in doc:
        if t.pos_ == "NOUN":
            if any(t.text in e for e in nomeadas):
                continue
            if any(t.text in c for c in compostas):
                continue
            substantivos.add(t.lemma_)

    nomeadas = {normalize_entity(e) for e in nomeadas}
    comp_lem = {normalize_entity(e) for e in comp_lem}
    substantivos = {normalize_entity(e) for e in substantivos}

    termos = list(nomeadas | comp_lem | substantivos)

    resolved = {t: (wiki.page(t).fullurl if wiki.page(t).exists() else "") for t in termos}

    return nomeadas, compostas, substantivos, resolved, termos

## Extração de relações semânticas

In [ ]:
# ------------------------------------------------------
#              EXTRAÇÃO DE RELAÇÕES (TRIPLETAS)
# ------------------------------------------------------

def normalize_relation(prep):
    mapa = {
        "de": "pertence a",
        "do": "pertence a",
        "da": "pertence a",
        "dos": "pertence a",
        "das": "pertence a",
        "em": "localizado em",
        "no": "localizado em",
        "na": "localizado em",
        "ao": "direcionado a",
        "a": "direcionado a"
    }
    return mapa.get(prep, prep)


def extract_svo_robust(doc, token_map):
    triplets = []

    verbs = [t for t in doc if t.pos_ in ("VERB","AUX")]
    if not verbs:
        return triplets

    root = None
    for v in verbs:
        if v.dep_ == "ROOT":
            root = v
            break
    if root is None:
        root = verbs[0]

    verbo = root

    # Cópula ("ser", "estar", ...)
    if verbo.lemma_.lower() in COPULAS:
        sujeitos = [w for w in verbo.lefts if w.dep_ in ("nsubj","nsubj:pass")]
        attrs = [w for w in verbo.rights if w.dep_ in ("attr","acomp","xcomp","obj")]
        if sujeitos and attrs:
            s = span_text(sujeitos[0], token_map)
            o = span_text(attrs[0], token_map)
            if s != o:
                triplets.append((s, verbo.lemma_, o))
            return triplets

    # Verbo normal
    sujeitos = [w for w in verbo.lefts if w.dep_ in ("nsubj","nsubj:pass")]
    objetos = [w for w in verbo.rights if w.dep_ in ("obj","iobj","obl")]

    if sujeitos and objetos:
        for s_tok in sujeitos:
            for o_tok in objetos:
                s = span_text(s_tok, token_map)
                o = span_text(o_tok, token_map)
                if s != o:
                    triplets.append((s, verbo.lemma_, o))
        return triplets

    # Fallback: procurar NOUN antes e depois
    toks = list(doc)
    idx = verbo.i

    left_noun = None
    for i in range(idx-1, -1, -1):
        if toks[i].pos_ in ("NOUN","PROPN"):
            left_noun = toks[i]
            break

    right_noun = None
    for i in range(idx+1, len(toks)):
        if toks[i].pos_ in ("NOUN","PROPN"):
            right_noun = toks[i]
            break

    if left_noun and right_noun:
        s = span_text(left_noun, token_map)
        o = span_text(right_noun, token_map)
        if s != o:
            triplets.append((s, verbo.lemma_, o))

    return triplets


def extract_nominal_relations(doc, token_map):
    rels = []
    for token in doc:
        if token.pos_ in ("NOUN","PROPN"):
            for child in token.children:
                if child.pos_ in ("NOUN","PROPN") and child.dep_ in ("nmod","obl","nmod:poss"):
                    preps = [w.text for w in child.children if w.dep_ == "case"]
                    if not preps:
                        continue
                    prep = preps[0]
                    A = span_text(token, token_map)
                    B = span_text(child, token_map)
                    if A != B:
                        rels.append((A, normalize_relation(prep), B))
    return rels

## Construção do KG

In [ ]:
#retorna o grafo de conhecimento do texto, suas entidades e suas relações semânticas
def construir_kg(texto):
    doc = nlp(texto)

    # entidades
    nomeadas, compostas, subst, resolved, termos = extrair_entidades(doc)

    # token map
    token_map = build_token_map(doc, compostas)

    # relações
    svo = extract_svo_robust(doc, token_map)
    nom = extract_nominal_relations(doc, token_map)

    tripletas = []
    for t in svo + nom:
        if t not in tripletas:
            tripletas.append(t)

    #grafo
    G = nx.DiGraph()
    if not tripletas:
        for t in termos:
            G.add_node(t)

    for s,r,o in tripletas:
        s = s.lower().strip()
        o = o.lower().strip()
        G.add_edge(s, o, label=r)

    return G, termos, tripletas

## Expansão de consulta

In [ ]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0 Safari/537.36"
    )
}

STOP_PATTERNS = [
    r"^lista de", r"^categoria", r"^portal", r"^predefinição",
    r"^anexo", r"^história", r"^ficheiro", r"^ajuda",
]

IGNORAR_TIPOS = [
    "século", "ano", "anos",
    "américa", "méxico", "espanha",
    "maias", "astecas", "caribe",
    "europa", "américa do sul", "américa central",
]


def eh_ruido(t):
    t = t.strip().lower()

    # 1. Bloquear namespaces da Wikipédia
    if any(t.startswith(prefix) for prefix in [
        "especial:", "wikipédia:", "Wikcionário", "ajuda:", "ficheiro:", "file:",
        "template:", "predefinição:", "categoria:", "portal:",
        "anexo:", "discussão:", "user:", "usuário:"
    ]):
        return True

    # 2. Remover termos técnicos comuns em links internos
    if t in ["case-sensitive"]:
        return True

    # 3. Padrões já existentes
    for p in STOP_PATTERNS:
        if re.match(p, t):
            return True

    # 4. Textos grandes demais
    if len(t.split()) > 4:
        return True

    # 5. Contém números
    if re.search(r"\d", t):
        return True

    # 6. Tipos irrelevantes definidos por você
    for bad in IGNORAR_TIPOS:
        if bad in t:
            return True

    return False

def baixar_html(termo):
    url = f"https://pt.wikipedia.org/wiki/{termo.replace(' ', '_')}"
    r = requests.get(url, headers=HEADERS)
    return r.text


def links_em_ordem(termo):
    html = baixar_html(termo)
    soup = BeautifulSoup(html, "html.parser")

    content = soup.find("div", id="mw-content-text")
    if content is None:
        return []

    links = []

    intro_parts = []
    for elem in content.children:
        if getattr(elem, "name", None) == "h2":
            break
        intro_parts.append(elem)

    intro_html = BeautifulSoup("".join(str(e) for e in intro_parts), "html.parser")

    for a in intro_html.find_all("a", href=True):
        href = a["href"]

        if not href.startswith("/wiki/"):
            continue

        title = href.replace("/wiki/", "")
        title = urllib.parse.unquote(title)
        title = title.replace("_", " ")
        title = re.sub(r"#.*", "", title)

        if not eh_ruido(title):
            if title not in links:
                links.append(title)

    return links


def filtrar_por_pos(termos):
    """Retém apenas substantivos e nomes próprios."""
    filtrados = []
    for t in termos:
        doc = nlp(t)
        # Considera o token principal (head)
        token = doc[0]
        if token.pos_ in ("NOUN", "PROPN"):
            filtrados.append(t)
    return filtrados


def expandir_consulta_kg(termos, tripletas, max_exp=20):
    expandidos = []
    G = nx.DiGraph()

    for s,r,o in tripletas:
        G.add_edge(s, o, label=r)

    for termo in termos:
        if termo not in expandidos:
            expandidos.append(termo)
            G.add_node(termo)

        links = links_em_ordem(termo)

        # filtragem por POS
        filtrados = filtrar_por_pos(links)

        for lk in filtrados[:max_exp]:
            if lk not in expandidos:
                expandidos.append(lk)
                G.add_node(lk)
                G.add_edge(termo, lk, label="expands_to")

    return G

## União dos grafos

In [ ]:
def unir_grafos(G_query, G_doc):
    """
    Une dois grafos NetworkX mantendo:
    - nós
    - arestas
    - atributos das arestas (label)
    Retorna o grafo combinado.
    """

    G_merged = nx.DiGraph()

    # Adiciona grafos
    G_merged.update(G_query)
    G_merged.update(G_doc)

    return G_merged

## Similaridade semântica

In [ ]:
def similaridade_entidades(G, e1, e2, max_dist=3):
    """
    Similaridade estrutural baseada em distância geodésica no grafo.
    Valores próximos de 1 quando e1 e e2 estão perto.
    """
    try:
        d = nx.shortest_path_length(G, e1, e2)
    except nx.NetworkXNoPath:
        return 0.0

    if d > max_dist:
        return 0.0

    # Similaridade decai com a distância
    return 1 - (d / max_dist)

def similaridade_docs(G_merged, G1, G2): # sendo G_merged a união de G1 e G2
    ents1 = list(G1.nodes)
    ents2 = list(G2.nodes)

    if not ents1 or not ents2:
        return 0.0

    score = []
    for e1 in ents1:
        for e2 in ents2:
            score.append(similaridade_entidades(G_merged.to_undirected(), e1, e2))

    return sum(score) / len(score)

def similaridade_docs_PR(G_merged, G1, G2):
    """
    Similaridade entre consulta e documento usando Personalized PageRank.
    Isso evita que componentes desconectadas zerem a similaridade.
    """
    G = G_merged.to_undirected()

    ents_q = list(G1.nodes)
    ents_d = list(G2.nodes)

    if not ents_q or not ents_d:
        return 0.0

    # Personalização: todo o PageRank começa na consulta
    personalization = {
        node: (1.0 / len(ents_q)) if node in ents_q else 0.0
        for node in G.nodes()
    }

    pr = nx.pagerank(G, personalization=personalization)

    num = 0.0
    den = 0.0

    for e1 in ents_q:
        for e2 in ents_d:
            sim = similaridade_entidades(G, e1, e2)

            peso = pr.get(e1, 0) * pr.get(e2, 0)

            num += peso * sim
            den += peso

    if den == 0:
        return 0.0

    return num / den

## BM25

In [ ]:
stopwords = ["a","o","e","é","de","do","da","em", "no","na","ser", "são","dos","com","como","ele", "eles", "ela", "elas", "os", "as","ao","para", "por", "pelo",
             "um", "uns", "uma", "umas", "que", "outro", "esse", "essa", "desse", "dessa", "desses", "dessas", "esses", "essas", "aquilo", "aquele", "aqueles",
             "aquela", "aquelas"]

In [ ]:
def normalizar_documentos(docs, stopwords):
    docs_normalizados = []

    # 2. Normaliza cada documento da lista
    for texto in docs:
        # quebra em tokens \w+ e lowercase
        tokens = re.findall(r'\w+', texto.lower(), flags=re.UNICODE)

        # remove stopwords
        tokens_filtrados = [t for t in tokens if t not in stopwords]

        docs_normalizados.append(tokens_filtrados)

    return docs_normalizados

def stemming(docs):
    stemmer = SnowballStemmer("portuguese")
    stems = []

    for i, doc in enumerate(docs):
        for j, palavra in enumerate(doc):
            s = stemmer.stem(palavra)
            doc[j] = s                   # CORREÇÃO AQUI
            if s not in stems:
                stems.append(s)

    return stems

def stemming_lista(lista):
    stemmer = SnowballStemmer("portuguese")
    return [stemmer.stem(p) for p in lista]

#calcula a matriz de frequências
def calcula_matriz_frequencia(docs, stems):
    dict_frequencias = {}
    dict_frequencias[""] = stems

    for i, doc in enumerate(docs):
        frequencias = []
        for stem in stems:
            frequencias.append(doc.count(stem))
        dict_frequencias[f"documento {i+1}"] = frequencias

    df_frequencias = pd.DataFrame(dict_frequencias)
    df_frequencias.set_index("", inplace=True)

    return df_frequencias

In [ ]:
def bm25(consulta, stems, df_frequencias, k1=1.0, b=0.75):
    N = df_frequencias.shape[1]  # número de documentos
    doc_lengths = df_frequencias.sum(axis=0).values
    avgdl = doc_lengths.mean()

    scores = [0.0 for _ in range(N)]

    for termo in consulta:
        if termo not in stems:
            continue

        n_t = sum(1 for j in range(N)
                  if df_frequencias.loc[termo, f'documento {j+1}'] > 0)

        if n_t == 0:
            continue

        idf = math.log((N - n_t + 0.5) / (n_t + 0.5))

        for j in range(N):
            f_tj = df_frequencias.loc[termo, f'documento {j+1}']
            if f_tj == 0:
                continue

            dl = doc_lengths[j]
            denom = f_tj + k1 * (1 - b + b * (dl / avgdl))
            score = float(idf * (f_tj * (k1 + 1)) / denom)
            scores[j] += score

    # NORMALIZA AQUI (correto)
    min_s = min(scores)
    max_s = max(scores)

    if max_s - min_s == 0:
        scores_norm = [0 for _ in scores]
    else:
        scores_norm = [(x - min_s) / (max_s - min_s) for x in scores]

    return scores_norm

## Métricas de performance

In [ ]:
def Score(ranking_BM25, ranking_KG, alpha=0.85):
    scores = []
    for i in range(len(ranking_BM25)):
        scores.append(ranking_BM25[i] + alpha * ranking_KG[i])
    return scores

def calcular_precisao_recall(ranking, relevantes):
    ranking_sorted = sorted(range(len(ranking)), key=lambda x: ranking[x], reverse=True)

    recuperados = ranking_sorted
    recuperados_relevantes = [doc for doc in ranking_sorted if ranking[doc] > 0 and doc in relevantes]

    tp = len(recuperados_relevantes)
    fp = len(ranking) - tp
    fn = len(relevantes) - tp

    precisao = tp / len(ranking) if len(ranking) > 0 else 0
    recall = tp / len(relevantes) if len(relevantes) > 0 else 0

    return precisao, recall

def calcular_MAP(ranking, relevantes):
    ranking_sorted = sorted(range(len(ranking)), key=lambda x: ranking[x], reverse=True)

    acertos = 0
    precisions = []

    for i, doc_id in enumerate(ranking_sorted, start=1):
        if doc_id in relevantes:
            acertos += 1
            precisions.append(acertos / i)

    if len(relevantes) == 0:
        return 0.0

    return sum(precisions) / len(relevantes)

def calcular_nDCG(ranking, relevantes, K=10):
    ranking_sorted = sorted(range(len(ranking)), key=lambda x: ranking[x], reverse=True)
    topK = ranking_sorted[:K]

    dcg = 0.0
    for i, doc_id in enumerate(topK, start=1):
        rel = 1 if doc_id in relevantes else 0
        dcg += rel / math.log2(i + 1)

    ideal_rels = [1] * min(len(relevantes), K)
    idcg = sum(r / math.log2(i + 1) for i, r in enumerate(ideal_rels, start=1))

    if idcg == 0:
        return 0.0

    return dcg / idcg

## Conjuntos de teste

In [ ]:
def limpar(texto):
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

In [ ]:
p = wiki.page("Período do Terror")
p.exists()

True

In [ ]:
Saude = {"consulta": "cigarro causa câncer de pulmão",
         "títulos": ["cigarro",
                    "câncer de pulmão",
                    "bronquite",
                    "asma",
                    "doenças respiratórias",
                    "computador"],
         "relevantes": [0, 1]
         }

Historia = {"consulta": "Revolução Francesa",
            "títulos": ["Período do Terror",
                        "Napoleão Bonaparte",
                        "Iluminismo",
                        "Queda da Bastilha",
                        "Fotossíntese",
                        "Guerras Napoleônicas"],
            "relevantes": [0, 2, 3]
            }

Tecnologia = {"consulta": "inovação em saúde digital e telemedicina",
            "títulos": ["Saúde digital",
                        "Genómica",
                        "Programa Genomas Brasil",
                        "Inteligência artificial na medicina",
                        "Telemedicina",
                        "Magna Carta",
                        "Complexo Econômico-Industrial da Saúde",
                        "Saúde pública",
                        "Política de saúde"],
            "relevantes": [1, 2, 3, 4]
            }

Ciencia = {"consulta": "O efeito de um buraco negro no espaço-tempo",
         "títulos": ["Horizonte de eventos",
                    "Galáxia",
                    "Relatividade geral",
                    "Guerra Fria",
                    "Espaço-tempo",
                    "Singularidade gravitacional",
                    "Acreção"],
         "relevantes": [0, 2, 4, 5]
         }

Medicina = {"consulta": "Diabetes mellitus tipo 2",
         "títulos": ["Resistência à insulina",
                    "Obesidade",
                    "Hiperglicemia",
                    "Metformina",
                    "Revolução industrial"],
         "relevantes": [0, 2, 3]
         }

Biologia = {"consulta": "Seleção natural",
         "títulos": ["Darwinismo",
                    "Tabela periódica",
                    "Teoria da evolução",
                    "Adaptação biológica",
                    "Pressão seletiva",
                    "União Soviética"],
         "relevantes": [0, 2, 3, 4]
         }

Filosofia = {"consulta": "A influência de Platão na filosofia ocidental",
         "títulos": ["Grécia antiga",
                    "Sócrates",
                    "Amor platônico",
                    "Platonismo",
                    "bronquite",
                    "Iluminismo",
                    "Friedrich Nietzsche",
                    "Aristóteles"],
         "relevantes": [1, 3, 7]
         }

Literatura = {"consulta": "William Shakespeare",
         "títulos": ["Poema",
                    "Romeu e Julieta",
                    "Hamlet",
                    "Teatro",
                    "Dramaturgia"],
         "relevantes": [1, 2]
         }

Geologia = {"consulta": "A dinâmica das placas tectónicas e sua influência nos terremotos",
    "títulos": ["Tectônica de placas",
                "Terremoto",
                "Limite convergente",
                "Vulcão",
                "Deriva continental",
                "Deserto do Saara",
                "Geofísica"],
    "relevantes": [0, 1, 2, 4]
}

MachadoAssis = {"consulta": "vida e obra de Machado de Assis",
    "títulos": ["Machado de Assis",
                "Dom Casmurro",
                "Realismo (literatura)",
                "Clarice Lispector",
                "Academia Brasileira de Letras",
                "Modernismo brasileiro",
                "câncer de pulmão"],
    "relevantes": [0, 1, 2, 4]
}

GuerraFria = {"consulta": "Conflitos políticos da Guerra Fria",
    "títulos": ["Guerra Fria",
                "OTAN",
                "Geofísica",
                "Pacto de Varsóvia",
                "Bloco ocidental",
                "Queda do Muro de Berlim",
                "Doutrina Truman",
                "Guerra da Coreia"],
    "relevantes": [0, 1, 3, 5, 6]
}

InteligenciaArtificial = {"consulta": "Fundamentos da inteligência artificial",
    "títulos": ["Inteligência artificial",
                "Aprendizado de máquina",
                "Rede neural artificial",
                "Robótica",
                "Alan Turing",
                "Ética na inteligência artificial",
                "Ruptura sino-soviética",
                "Computação clássica"],
    "relevantes": [0, 1, 2, 5]
}

MutacaoGenetica = {"consulta": "Mutações no material genético",
    "títulos": ["Mutação",
                "DNA",
                "Evolução",
                "Engenharia genética",
                "Doença genética",
                "Cruzamento genético",
                "Genética",
                "Dom Casmurro"],
    "relevantes": [0, 2, 3, 4]
}

UniaoEuropeia = {"consulta": "Estrutura política da União Europeia",
    "títulos": ["União Europeia",
                "Tratado de Maastricht",
                "Euro",
                "Comunidades Europeias",
                "Tratado de Bruxelas",
                "OTAN",
                "Mercado comum da União Europeia",
                "Brexit",
                "Robótica"],
    "relevantes": [0, 1, 2, 4, 6, 7]
}

Psicologia = {"consulta": "Os mecanismos cognitivos envolvidos na formação de memória de longo prazo",
    "títulos": ["Memória",
                "Hipocampo",
                "Plasticidade sináptica",
                "Comportamento animal",
                "Cognição",
                "Transtorno bipolar",
                "Córtex pré-frontal"],
    "relevantes": [0, 1, 2, 4, 6]
}

SegundaGuerra = {
    "consulta": "As causas da Segunda Guerra Mundial",
    "títulos": [
        "Tratado de Versalhes",
        "Adolf Hitler",
        "Primeira Guerra Mundial",
        "Revolução Francesa",
        "Nazismo",
        "Holocausto",
        "Império Romano"
    ],
    "relevantes": [0, 1, 2, 4]
}

Genetica = {
    "consulta": "O funcionamento do DNA nas células",
    "títulos": [
        "Ácido desoxirribonucleico",
        "ARN mensageiro",
        "Cromossomo",
        "Fotossíntese",
        "Núcleo celular",
        "Membrana plasmática",
        "Mutação"
    ],
    "relevantes": [0, 1, 2, 4, 6]
}

Inflacao = {
    "consulta": "As causas da inflação em economias modernas",
    "títulos": [
        "Inflação",
        "Lei da oferta e da procura",
        "Política monetária",
        "Produto interno bruto",
        "Banco Central",
        "Desemprego",
        "Finanças públicas"
    ],
    "relevantes": [0, 2, 4, 6]
}

TabelaPeriodica = {
    "consulta": "A organização da tabela periódica dos elementos",
    "títulos": [
        "Tabela periódica",
        "Elemento (química)",
        "Ligação química",
        "Isótopo",
        "Modelo atômico",
        "Reação química"
    ],
    "relevantes": [0, 1, 4]
}

MitologiaGrega = {
    "consulta": "Os principais deuses da mitologia grega",
    "títulos": [
        "Zeus",
        "Atena",
        "Poseidon",
        "Mitologia nórdica",
        "Hércules",
        "Olimpo",
        "Hades"
    ],
    "relevantes": [0, 1, 2, 6]
}


Temas_titulos = [Saude, Historia, Tecnologia, Ciencia, Medicina, Biologia, Filosofia,
                 Literatura, Geologia, MachadoAssis, GuerraFria, InteligenciaArtificial,
                 MutacaoGenetica, UniaoEuropeia, Psicologia, SegundaGuerra, Genetica,
                 Inflacao, TabelaPeriodica, MitologiaGrega]

In [ ]:
def wikipedia_paragrafos(titulos):
    docs = []
    for t in titulos:
        p = wiki.page(t)
        if p.exists():
            intro = p.summary.split("\n")[0]
            docs.append(limpar(intro))
        else:
            docs.append("")  # opcional: evita perder ordem
    return docs

def substituir_titulos_por_docs(temas):
    novos_temas = []
    for tema in temas:
        titulos = tema.get("títulos", [])
        docs = wikipedia_paragrafos(titulos)

        # criar nova estrutura
        novo_tema = tema.copy()
        del novo_tema["títulos"]
        novo_tema["documentos"] = docs

        novos_temas.append(novo_tema)

    return novos_temas

In [ ]:
Temas = substituir_titulos_por_docs(Temas_titulos)

## Testes

In [ ]:
def avaliar_ranking_BM25(documentos, consulta, relevantes):
    docs = normalizar_documentos(documentos, stopwords)
    stems = stemming(docs)
    consulta_stem = stemming_lista(consulta.split())

    df_frequencias = calcula_matriz_frequencia(docs, stems)

    ranking_BM25 = bm25(consulta_stem, stems, df_frequencias)
    print("----------- Ranking BM25 -----------")
    print(f"q = {consulta}")
    for i, ranking in enumerate(ranking_BM25):
        print(f"BM25(q, d{i}): {ranking}")
    p, r = calcular_precisao_recall(ranking_BM25, relevantes)
    MAP10  = calcular_MAP(ranking_BM25, relevantes)
    nDCG10 = calcular_nDCG(ranking_BM25, relevantes)

    print("--------- Métricas de performance -------")
    print("Precisão:", p)
    print("Recall:", r)
    print("MAP:", MAP10)
    print("nDCG:", nDCG10)

    return ranking_BM25, p, r, MAP10, nDCG10

In [ ]:
def avaliar_ranking_BM25_KG(documentos, consulta, relevantes, ranking_BM25):
    G_consulta, ents_c, rels_c = construir_kg(consulta)
    consulta_exp = expandir_consulta_kg(ents_c, rels_c, max_exp=20)

    ranking_KG = []
    for i, doc in enumerate(documentos):
        G_doc, ents, rels = construir_kg(doc)

        G_merged = unir_grafos(consulta_exp, G_doc)
        sim = similaridade_docs_PR(G_merged, consulta_exp, G_doc)
        ranking_KG.append(sim)

    print("----------- Sim KG -----------")
    print(f"q = {consulta}")
    for i, sim in enumerate(ranking_KG):
        print(f"Sim(q, d{i}) = ", sim)
    scores = Score(ranking_BM25, ranking_KG)
    print("----------- Ranking BM25 + Sim KG -----------")
    for i, score in enumerate(scores):
        print(f"Score(q, d{i}) = ", score)
    p, r = calcular_precisao_recall(scores, relevantes)
    MAP10  = calcular_MAP(scores, relevantes)
    nDCG10 = calcular_nDCG(scores, relevantes)

    print("--------- Métricas de performance -------")
    print("Precisão:", p)
    print("Recall:", r)
    print("MAP:", MAP10)
    print("nDCG:", nDCG10)

    return scores, p, r, MAP10, nDCG10

In [ ]:
precisao_BM25 = []
recall_BM25 = []
MAP_BM25 = []
nDCG_BM25 = []

precisao_BM25_KG = []
recall_BM25_KG = []
MAP_BM25_KG = []
nDCG_BM25_KG = []

### Tema 1

In [ ]:
# ------------Tema 1: Saúde --------------------
consulta1 = Temas[0]["consulta"]
documentos1 = Temas[0]["documentos"]
relevantes1 = Temas[0]["relevantes"]

In [ ]:
ranking_BM25_1, p_BM25_1, r_BM25_1, MAP10_BM25_1, nDCG10_BM25_1 = avaliar_ranking_BM25(documentos1, consulta1, relevantes1)

----------- Ranking BM25 -----------
q = cigarro causa câncer de pulmão
BM25(q, d0): 0.39441756320768195
BM25(q, d1): 1.0
BM25(q, d2): 0.0
BM25(q, d3): 0.0
BM25(q, d4): 0.0
BM25(q, d5): 0.0
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25.append(p_BM25_1)
recall_BM25.append(r_BM25_1)
MAP_BM25.append(MAP10_BM25_1)
nDCG_BM25.append(nDCG10_BM25_1)

In [ ]:
ranking_BM25_KG_1, p_KG_1, r_KG_1, MAP10_KG_1, nDCG10_KG_1 = avaliar_ranking_BM25_KG(documentos1, consulta1, relevantes1, ranking_BM25_1)

----------- Sim KG -----------
q = cigarro causa câncer de pulmão
Sim(q, d0) =  0.6073382628716338
Sim(q, d1) =  0.646009516944061
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.9106550866485708
Score(q, d1) =  1.5491080894024518
Score(q, d2) =  0.0
Score(q, d3) =  0.0
Score(q, d4) =  0.0
Score(q, d5) =  0.0
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25_KG.append(p_KG_1)
recall_BM25_KG.append(r_KG_1)
MAP_BM25_KG.append(MAP10_KG_1)
nDCG_BM25_KG.append(nDCG10_KG_1)

### Tema 2

In [ ]:
# ------------Tema 2: história --------------------
consulta2 = Temas[1]["consulta"]
documentos2 = Temas[1]["documentos"]
relevantes2 = Temas[1]["relevantes"]

In [ ]:
ranking_BM25_2, p_BM25_2, r_BM25_2, MAP10_BM25_2, nDCG10_BM25_2 = avaliar_ranking_BM25(documentos2, consulta2, relevantes2)

----------- Ranking BM25 -----------
q = Revolução Francesa
BM25(q, d0): 0.13196570728720133
BM25(q, d1): 0.13858049492448202
BM25(q, d2): 1.0
BM25(q, d3): 0.0
BM25(q, d4): 1.0
BM25(q, d5): 0.430265273311897
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 0.6666666666666666
MAP: 0.6333333333333333
nDCG: 0.8179809735263114


In [ ]:
precisao_BM25.append(p_BM25_2)
recall_BM25.append(r_BM25_2)
MAP_BM25.append(MAP10_BM25_2)
nDCG_BM25.append(nDCG10_BM25_2)

In [ ]:
ranking_BM25_KG_2, p_KG_2, r_KG_2, MAP10_KG_2, nDCG10_KG_2 = avaliar_ranking_BM25_KG(documentos2, consulta2, relevantes2, ranking_BM25_2)

----------- Sim KG -----------
q = Revolução Francesa
Sim(q, d0) =  0.7595588927260362
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.8110660922867423
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.7775907661043321
Score(q, d1) =  0.13858049492448202
Score(q, d2) =  1.0
Score(q, d3) =  0.689406178443731
Score(q, d4) =  1.0
Score(q, d5) =  0.430265273311897
--------- Métricas de performance -------
Precisão: 0.5
Recall: 1.0
MAP: 0.8055555555555555
nDCG: 0.9060254355346823


In [ ]:
precisao_BM25_KG.append(p_KG_2)
recall_BM25_KG.append(r_KG_2)
MAP_BM25_KG.append(MAP10_KG_2)
nDCG_BM25_KG.append(nDCG10_KG_2)

### Tema 3

In [ ]:
# ------------Tema 3: tecnologia --------------------
consulta3 = Temas[2]["consulta"]
documentos3 = Temas[2]["documentos"]
relevantes3 = Temas[2]["relevantes"]

In [ ]:
ranking_BM25_3, p_BM25_3, r_BM25_3, MAP10_BM25_3, nDCG10_BM25_3 = avaliar_ranking_BM25(documentos3, consulta3, relevantes3)

----------- Ranking BM25 -----------
q = inovação em saúde digital e telemedicina
BM25(q, d0): 1.0
BM25(q, d1): 0.33470680543013587
BM25(q, d2): 0.07002445404050879
BM25(q, d3): 0.33470680543013587
BM25(q, d4): 0.4983958568528896
BM25(q, d5): 0.33470680543013587
BM25(q, d6): 0.015886583383679678
BM25(q, d7): 0.03955081507635418
BM25(q, d8): 0.0
--------- Métricas de performance -------
Precisão: 0.4444444444444444
Recall: 1.0
MAP: 0.6458333333333333
nDCG: 0.7486761295186739


In [ ]:
precisao_BM25.append(p_BM25_3)
recall_BM25.append(r_BM25_3)
MAP_BM25.append(MAP10_BM25_3)
nDCG_BM25.append(nDCG10_BM25_3)

In [ ]:
ranking_BM25_KG_3, p_KG_3, r_KG_3, MAP10_KG_3, nDCG10_KG_3 = avaliar_ranking_BM25_KG(documentos3, consulta3, relevantes3, ranking_BM25_3)

----------- Sim KG -----------
q = inovação em saúde digital e telemedicina
Sim(q, d0) =  0.41606614614482035
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.2373421989673534
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
Sim(q, d7) =  0.0
Sim(q, d8) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.3536562242230974
Score(q, d1) =  0.33470680543013587
Score(q, d2) =  0.07002445404050879
Score(q, d3) =  0.33470680543013587
Score(q, d4) =  0.70013672597514
Score(q, d5) =  0.33470680543013587
Score(q, d6) =  0.015886583383679678
Score(q, d7) =  0.03955081507635418
Score(q, d8) =  0.0
--------- Métricas de performance -------
Precisão: 0.4444444444444444
Recall: 1.0
MAP: 0.6458333333333333
nDCG: 0.7486761295186739


In [ ]:
precisao_BM25_KG.append(p_KG_3)
recall_BM25_KG.append(r_KG_3)
MAP_BM25_KG.append(MAP10_KG_3)
nDCG_BM25_KG.append(nDCG10_KG_3)

### Tema 4

In [ ]:
# ------------Tema 3: ciência --------------------
consulta4 = Temas[3]["consulta"]
documentos4 = Temas[3]["documentos"]
relevantes4 = Temas[3]["relevantes"]

In [ ]:
ranking_BM25_4, p_BM25_4, r_BM25_4, MAP10_BM25_4, nDCG10_BM25_4 = avaliar_ranking_BM25(documentos4, consulta4, relevantes4)

----------- Ranking BM25 -----------
q = O efeito de um buraco negro no espaço-tempo
BM25(q, d0): 1.0
BM25(q, d1): 0.0
BM25(q, d2): 0.0
BM25(q, d3): 0.0
BM25(q, d4): 0.0
BM25(q, d5): 0.0
BM25(q, d6): 0.0
--------- Métricas de performance -------
Precisão: 0.14285714285714285
Recall: 0.25
MAP: 0.7333333333333333
nDCG: 0.8756458727267334


In [ ]:
precisao_BM25.append(p_BM25_4)
recall_BM25.append(r_BM25_4)
MAP_BM25.append(MAP10_BM25_4)
nDCG_BM25.append(nDCG10_BM25_4)

In [ ]:
ranking_BM25_KG_4, p_KG_4, r_KG_4, MAP10_KG_4, nDCG10_KG_4 = avaliar_ranking_BM25_KG(documentos4, consulta4, relevantes4, ranking_BM25_4)

----------- Sim KG -----------
q = O efeito de um buraco negro no espaço-tempo
Sim(q, d0) =  0.5880317884476525
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.5829891337366778
Sim(q, d5) =  0.5830016271445178
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.4998270201805046
Score(q, d1) =  0.0
Score(q, d2) =  0.0
Score(q, d3) =  0.0
Score(q, d4) =  0.4955407636761761
Score(q, d5) =  0.4955513830728401
Score(q, d6) =  0.0
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.95
nDCG: 0.9828920819566879


In [ ]:
precisao_BM25_KG.append(p_KG_4)
recall_BM25_KG.append(r_KG_4)
MAP_BM25_KG.append(MAP10_KG_4)
nDCG_BM25_KG.append(nDCG10_KG_4)

### Tema 5

In [ ]:
# ------------Tema 5: biologia --------------------
consulta5 = Temas[4]["consulta"]
documentos5 = Temas[4]["documentos"]
relevantes5 = Temas[4]["relevantes"]

In [ ]:
ranking_BM25_5, p_BM25_5, r_BM25_5, MAP10_BM25_5, nDCG10_BM25_5 = avaliar_ranking_BM25(documentos5, consulta5, relevantes5)

----------- Ranking BM25 -----------
q = Diabetes mellitus tipo 2
BM25(q, d0): 0.0
BM25(q, d1): 0.6705882352941177
BM25(q, d2): 0.0
BM25(q, d3): 1.0
BM25(q, d4): 0.0
--------- Métricas de performance -------
Precisão: 0.2
Recall: 0.3333333333333333
MAP: 0.8055555555555555
nDCG: 0.9060254355346823


In [ ]:
precisao_BM25.append(p_BM25_5)
recall_BM25.append(r_BM25_5)
MAP_BM25.append(MAP10_BM25_5)
nDCG_BM25.append(nDCG10_BM25_5)

In [ ]:
ranking_BM25_KG_5, p_KG_5, r_KG_5, MAP10_KG_5, nDCG10_KG_5 = avaliar_ranking_BM25_KG(documentos5, consulta5, relevantes5, ranking_BM25_5)

----------- Sim KG -----------
q = Diabetes mellitus tipo 2
Sim(q, d0) =  0.0
Sim(q, d1) =  0.8037257424077938
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.0
Score(q, d1) =  1.3537551163407424
Score(q, d2) =  0.0
Score(q, d3) =  1.0
Score(q, d4) =  0.0
--------- Métricas de performance -------
Precisão: 0.2
Recall: 0.3333333333333333
MAP: 0.6388888888888888
nDCG: 0.7328286204777911


In [ ]:
precisao_BM25_KG.append(p_KG_5)
recall_BM25_KG.append(r_KG_5)
MAP_BM25_KG.append(MAP10_KG_5)
nDCG_BM25_KG.append(nDCG10_KG_5)

### Tema 6

In [ ]:
# ------------Tema 6: biologia --------------------
consulta6 = Temas[5]["consulta"]
documentos6 = Temas[5]["documentos"]
relevantes6 = Temas[5]["relevantes"]

In [ ]:
ranking_BM25_6, p_BM25_6, r_BM25_6, MAP10_BM25_6, nDCG10_BM25_6 = avaliar_ranking_BM25(documentos6, consulta6, relevantes6)

----------- Ranking BM25 -----------
q = Seleção natural
BM25(q, d0): 1.0
BM25(q, d1): 0.0
BM25(q, d2): 0.7753510140405617
BM25(q, d3): 0.0
BM25(q, d4): 0.0
BM25(q, d5): 0.0
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 0.5
MAP: 0.8875
nDCG: 0.9558295932317544


In [ ]:
precisao_BM25.append(p_BM25_6)
recall_BM25.append(r_BM25_6)
MAP_BM25.append(MAP10_BM25_6)
nDCG_BM25.append(nDCG10_BM25_6)

In [ ]:
ranking_BM25_KG_6, p_KG_6, r_KG_6, MAP10_KG_6, nDCG10_KG_6 = avaliar_ranking_BM25_KG(documentos6, consulta6, relevantes6, ranking_BM25_6)

----------- Sim KG -----------
q = Seleção natural
Sim(q, d0) =  0.0
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.0
Score(q, d1) =  0.0
Score(q, d2) =  0.7753510140405617
Score(q, d3) =  0.0
Score(q, d4) =  0.0
Score(q, d5) =  0.0
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 0.5
MAP: 0.8875
nDCG: 0.9558295932317544


In [ ]:
precisao_BM25_KG.append(p_KG_6)
recall_BM25_KG.append(r_KG_6)
MAP_BM25_KG.append(MAP10_KG_6)
nDCG_BM25_KG.append(nDCG10_KG_6)

### Tema 7

In [ ]:
# ------------Tema 7: filosofia --------------------
consulta7 = Temas[6]["consulta"]
documentos7 = Temas[6]["documentos"]
relevantes7 = Temas[6]["relevantes"]

In [ ]:
ranking_BM25_7, p_BM25_7, r_BM25_7, MAP10_BM25_7, nDCG10_BM25_7 = avaliar_ranking_BM25(documentos7, consulta7, relevantes7)

----------- Ranking BM25 -----------
q = A influência de Platão na filosofia ocidental
BM25(q, d0): 0.0
BM25(q, d1): 1.0
BM25(q, d2): 0.0
BM25(q, d3): 0.42273312177152284
BM25(q, d4): 0.0
BM25(q, d5): 0.0
BM25(q, d6): 0.0
BM25(q, d7): 0.7804862766887565
--------- Métricas de performance -------
Precisão: 0.375
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25.append(p_BM25_7)
recall_BM25.append(r_BM25_7)
MAP_BM25.append(MAP10_BM25_7)
nDCG_BM25.append(nDCG10_BM25_7)

In [ ]:
ranking_BM25_KG_7, p_KG_7, r_KG_7, MAP10_KG_7, nDCG10_KG_7 = avaliar_ranking_BM25_KG(documentos7, consulta7, relevantes7, ranking_BM25_7)

----------- Sim KG -----------
q = A influência de Platão na filosofia ocidental
Sim(q, d0) =  0.0
Sim(q, d1) =  0.43508170631455007
Sim(q, d2) =  0.0
Sim(q, d3) =  0.31009865077430937
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
Sim(q, d7) =  0.32344627766964884
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.0
Score(q, d1) =  1.3698194503673675
Score(q, d2) =  0.0
Score(q, d3) =  0.6863169749296858
Score(q, d4) =  0.0
Score(q, d5) =  0.0
Score(q, d6) =  0.0
Score(q, d7) =  1.055415612707958
--------- Métricas de performance -------
Precisão: 0.375
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25_KG.append(p_KG_7)
recall_BM25_KG.append(r_KG_7)
MAP_BM25_KG.append(MAP10_KG_7)
nDCG_BM25_KG.append(nDCG10_KG_7)

### Tema 8

In [ ]:
# ------------Tema 8: Literatura --------------------
consulta8 = Temas[7]["consulta"]
documentos8 = Temas[7]["documentos"]
relevantes8 = Temas[7]["relevantes"]

In [ ]:
ranking_BM25_8, p_BM25_8, r_BM25_8, MAP10_BM25_8, nDCG10_BM25_8 = avaliar_ranking_BM25(documentos8, consulta8, relevantes8)

----------- Ranking BM25 -----------
q = William Shakespeare
BM25(q, d0): 0.0
BM25(q, d1): 1.0
BM25(q, d2): 0.7810219382692969
BM25(q, d3): 0.0
BM25(q, d4): 0.0
--------- Métricas de performance -------
Precisão: 0.4
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25.append(p_BM25_8)
recall_BM25.append(r_BM25_8)
MAP_BM25.append(MAP10_BM25_8)
nDCG_BM25.append(nDCG10_BM25_8)

In [ ]:
ranking_BM25_KG_8, p_KG_8, r_KG_8, MAP10_KG_8, nDCG10_KG_8 = avaliar_ranking_BM25_KG(documentos8, consulta8, relevantes8, ranking_BM25_8)

----------- Sim KG -----------
q = William Shakespeare
Sim(q, d0) =  0.0
Sim(q, d1) =  0.5781844070746653
Sim(q, d2) =  0.7166465911171113
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.0
Score(q, d1) =  1.4914567460134656
Score(q, d2) =  1.3901715407188413
Score(q, d3) =  0.0
Score(q, d4) =  0.0
--------- Métricas de performance -------
Precisão: 0.4
Recall: 1.0
MAP: 1.0
nDCG: 1.0


In [ ]:
precisao_BM25_KG.append(p_KG_8)
recall_BM25_KG.append(r_KG_8)
MAP_BM25_KG.append(MAP10_KG_8)
nDCG_BM25_KG.append(nDCG10_KG_8)

### Tema 9

In [ ]:
# ------------Tema 9: Historia 2 --------------------
consulta9 = Temas[8]["consulta"]
documentos9 = Temas[8]["documentos"]
relevantes9 = Temas[8]["relevantes"]

In [ ]:
ranking_BM25_9, p_BM25_9, r_BM25_9, MAP10_BM25_9, nDCG10_BM25_9 = avaliar_ranking_BM25(documentos9, consulta9, relevantes9)

----------- Ranking BM25 -----------
q = A dinâmica das placas tectónicas e sua influência nos terremotos
BM25(q, d0): 0.3838951451671118
BM25(q, d1): 0.8207262561007204
BM25(q, d2): 0.6083244306311727
BM25(q, d3): 0.0
BM25(q, d4): 0.6213426091895802
BM25(q, d5): 0.0
BM25(q, d6): 1.0
--------- Métricas de performance -------
Precisão: 0.5714285714285714
Recall: 1.0
MAP: 0.6791666666666667
nDCG: 0.7606395682357036


In [ ]:
precisao_BM25.append(p_BM25_9)
recall_BM25.append(r_BM25_9)
MAP_BM25.append(MAP10_BM25_9)
nDCG_BM25.append(nDCG10_BM25_9)

In [ ]:
ranking_BM25_KG_9, p_KG_9, r_KG_9, MAP10_KG_9, nDCG10_KG_9 = avaliar_ranking_BM25_KG(documentos9, consulta9, relevantes9, ranking_BM25_9)

----------- Sim KG -----------
q = A dinâmica das placas tectónicas e sua influência nos terremotos
Sim(q, d0) =  0.0
Sim(q, d1) =  0.027327178844495998
Sim(q, d2) =  0.1539929319839587
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.3838951451671118
Score(q, d1) =  0.8439543581185419
Score(q, d2) =  0.7392184228175376
Score(q, d3) =  0.0
Score(q, d4) =  0.6213426091895802
Score(q, d5) =  0.0
Score(q, d6) =  1.0
--------- Métricas de performance -------
Precisão: 0.5714285714285714
Recall: 1.0
MAP: 0.6791666666666667
nDCG: 0.7606395682357036


In [ ]:
precisao_BM25_KG.append(p_KG_9)
recall_BM25_KG.append(r_KG_9)
MAP_BM25_KG.append(MAP10_KG_9)
nDCG_BM25_KG.append(nDCG10_KG_9)

### Tema 10

In [ ]:
# ------------Tema 10: Historia 2 --------------------
consulta10 = Temas[9]["consulta"]
documentos10 = Temas[9]["documentos"]
relevantes10 = Temas[9]["relevantes"]

In [ ]:
ranking_BM25_10, p_BM25_10, r_BM25_10, MAP10_BM25_10, nDCG10_BM25_10 = avaliar_ranking_BM25(documentos10, consulta10, relevantes10)

----------- Ranking BM25 -----------
q = vida e obra de Machado de Assis
BM25(q, d0): 0.46454743915948304
BM25(q, d1): 0.5134165498537805
BM25(q, d2): 0.0
BM25(q, d3): 1.0
BM25(q, d4): 0.3934386900750899
BM25(q, d5): 0.0
BM25(q, d6): 0.0
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.6791666666666667
nDCG: 0.7606395682357036


In [ ]:
precisao_BM25.append(p_BM25_10)
recall_BM25.append(r_BM25_10)
MAP_BM25.append(MAP10_BM25_10)
nDCG_BM25.append(nDCG10_BM25_10)

In [ ]:
ranking_BM25_KG_10, p_KG_10, r_KG_10, MAP10_KG_10, nDCG10_KG_10 = avaliar_ranking_BM25_KG(documentos10, consulta10, relevantes10, ranking_BM25_10)

----------- Sim KG -----------
q = vida e obra de Machado de Assis
Sim(q, d0) =  0.010755047810132675
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.4736892297980958
Score(q, d1) =  0.5134165498537805
Score(q, d2) =  0.0
Score(q, d3) =  1.0
Score(q, d4) =  0.3934386900750899
Score(q, d5) =  0.0
Score(q, d6) =  0.0
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.6791666666666667
nDCG: 0.7606395682357036


In [ ]:
precisao_BM25_KG.append(p_KG_10)
recall_BM25_KG.append(r_KG_10)
MAP_BM25_KG.append(MAP10_KG_10)
nDCG_BM25_KG.append(nDCG10_KG_10)

### Tema 11

In [ ]:
# ------------Tema 11: Geopolítica 2 --------------------
consulta11 = Temas[10]["consulta"]
documentos11 = Temas[10]["documentos"]
relevantes11 = Temas[10]["relevantes"]

In [ ]:
ranking_BM25_11, p_BM25_11, r_BM25_11, MAP10_BM25_11, nDCG10_BM25_11 = avaliar_ranking_BM25(documentos11, consulta11, relevantes11)

----------- Ranking BM25 -----------
q = Conflitos políticos da Guerra Fria
BM25(q, d0): 0.3090354731971063
BM25(q, d1): 1.0
BM25(q, d2): 1.0
BM25(q, d3): 0.20190820010314606
BM25(q, d4): 0.09157616671558563
BM25(q, d5): 0.10185722576900748
BM25(q, d6): 0.0
BM25(q, d7): 0.40100638668473004
--------- Métricas de performance -------
Precisão: 0.5
Recall: 0.8
MAP: 0.6783333333333333
nDCG: 0.8442380676954223


In [ ]:
precisao_BM25.append(p_BM25_11)
recall_BM25.append(r_BM25_11)
MAP_BM25.append(MAP10_BM25_11)
nDCG_BM25.append(nDCG10_BM25_11)

In [ ]:
ranking_BM25_KG_11, p_KG_11, r_KG_11, MAP10_KG_11, nDCG10_KG_11 = avaliar_ranking_BM25_KG(documentos11, consulta11, relevantes11, ranking_BM25_11)

----------- Sim KG -----------
q = Conflitos políticos da Guerra Fria
Sim(q, d0) =  0.6913726065727172
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.7344237079688946
Sim(q, d5) =  0.6874355713064232
Sim(q, d6) =  0.7344265319962745
Sim(q, d7) =  0.7116336055011763
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.8967021887839159
Score(q, d1) =  1.0
Score(q, d2) =  1.0
Score(q, d3) =  0.20190820010314606
Score(q, d4) =  0.715836318489146
Score(q, d5) =  0.6861774613794671
Score(q, d6) =  0.6242625521968334
Score(q, d7) =  1.0058949513607298
--------- Métricas de performance -------
Precisão: 0.625
Recall: 1.0
MAP: 0.5392857142857143
nDCG: 0.7009124514018641


In [ ]:
precisao_BM25_KG.append(p_KG_11)
recall_BM25_KG.append(r_KG_11)
MAP_BM25_KG.append(MAP10_KG_11)
nDCG_BM25_KG.append(nDCG10_KG_11)

### Tema 12

In [ ]:
# ------------Tema 12: Inteligencia artificial 2 --------------------
consulta12 = Temas[11]["consulta"]
documentos12 = Temas[11]["documentos"]
relevantes12 = Temas[11]["relevantes"]

In [ ]:
ranking_BM25_12, p_BM25_12, r_BM25_12, MAP10_BM25_12, nDCG10_BM25_12 = avaliar_ranking_BM25(documentos12, consulta12, relevantes12)

----------- Ranking BM25 -----------
q = Fundamentos da inteligência artificial
BM25(q, d0): 0.08775137111517367
BM25(q, d1): 0.2962627178402337
BM25(q, d2): 1.0
BM25(q, d3): 0.2138194913346838
BM25(q, d4): 0.41695877149056915
BM25(q, d5): 0.0
BM25(q, d6): 1.0
BM25(q, d7): 1.0
--------- Métricas de performance -------
Precisão: 0.375
Recall: 0.75
MAP: 0.5821428571428572
nDCG: 0.7946775459209724


In [ ]:
precisao_BM25.append(p_BM25_12)
recall_BM25.append(r_BM25_12)
MAP_BM25.append(MAP10_BM25_12)
nDCG_BM25.append(nDCG10_BM25_12)

In [ ]:
ranking_BM25_KG_12, p_KG_12, r_KG_12, MAP10_KG_12, nDCG10_KG_12 = avaliar_ranking_BM25_KG(documentos12, consulta12, relevantes12, ranking_BM25_12)

----------- Sim KG -----------
q = Fundamentos da inteligência artificial
Sim(q, d0) =  0.0
Sim(q, d1) =  0.7833442521388252
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.7833576384679785
Sim(q, d6) =  0.0
Sim(q, d7) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.08775137111517367
Score(q, d1) =  0.9621053321582351
Score(q, d2) =  1.0
Score(q, d3) =  0.2138194913346838
Score(q, d4) =  0.41695877149056915
Score(q, d5) =  0.6658539926977817
Score(q, d6) =  1.0
Score(q, d7) =  1.0
--------- Métricas de performance -------
Precisão: 0.5
Recall: 1.0
MAP: 0.65
nDCG: 0.8326783988613894


In [ ]:
precisao_BM25_KG.append(p_KG_12)
recall_BM25_KG.append(r_KG_12)
MAP_BM25_KG.append(MAP10_KG_12)
nDCG_BM25_KG.append(nDCG10_KG_12)

### Tema 13

In [ ]:
# ------------Tema 13: Mutação genética --------------------
consulta13 = Temas[12]["consulta"]
documentos13 = Temas[12]["documentos"]
relevantes13 = Temas[12]["relevantes"]

In [ ]:
ranking_BM25_13, p_BM25_13, r_BM25_13, MAP10_BM25_13, nDCG10_BM25_13 = avaliar_ranking_BM25(documentos13, consulta13, relevantes13)

----------- Ranking BM25 -----------
q = Mutações no material genético
BM25(q, d0): 1.0
BM25(q, d1): 0.17913554761964676
BM25(q, d2): 0.1929460042106038
BM25(q, d3): 0.05194071486782902
BM25(q, d4): 0.19261373378172608
BM25(q, d5): 0.15514506200630987
BM25(q, d6): 0.0
BM25(q, d7): 0.5514424398841973
--------- Métricas de performance -------
Precisão: 0.5
Recall: 1.0
MAP: 0.7470238095238095
nDCG: 0.8838242945899706


In [ ]:
precisao_BM25.append(p_BM25_13)
recall_BM25.append(r_BM25_13)
MAP_BM25.append(MAP10_BM25_13)
nDCG_BM25.append(nDCG10_BM25_13)

In [ ]:
ranking_BM25_KG_13, p_KG_13, r_KG_13, MAP10_KG_13, nDCG10_KG_13 = avaliar_ranking_BM25_KG(documentos13, consulta13, relevantes13, ranking_BM25_13)

----------- Sim KG -----------
q = Mutações no material genético
Sim(q, d0) =  0.3912520112824942
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
Sim(q, d7) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.33256420959012
Score(q, d1) =  0.17913554761964676
Score(q, d2) =  0.1929460042106038
Score(q, d3) =  0.05194071486782902
Score(q, d4) =  0.19261373378172608
Score(q, d5) =  0.15514506200630987
Score(q, d6) =  0.0
Score(q, d7) =  0.5514424398841973
--------- Métricas de performance -------
Precisão: 0.5
Recall: 1.0
MAP: 0.7470238095238095
nDCG: 0.8838242945899706


In [ ]:
precisao_BM25_KG.append(p_KG_13)
recall_BM25_KG.append(r_KG_13)
MAP_BM25_KG.append(MAP10_KG_13)
nDCG_BM25_KG.append(nDCG10_KG_13)

### Tema 14

In [ ]:
# ------------Tema 14: União Europeia --------------------
consulta14 = Temas[13]["consulta"]
documentos14 = Temas[13]["documentos"]
relevantes14 = Temas[13]["relevantes"]

In [ ]:
ranking_BM25_14, p_BM25_14, r_BM25_14, MAP10_BM25_14, nDCG10_BM25_14 = avaliar_ranking_BM25(documentos14, consulta14, relevantes14)

----------- Ranking BM25 -----------
q = Estrutura política da União Europeia
BM25(q, d0): 0.36139273487149476
BM25(q, d1): 0.0
BM25(q, d2): 0.15594917172508874
BM25(q, d3): 0.441082848885326
BM25(q, d4): 0.5995099920793582
BM25(q, d5): 0.7472821849574904
BM25(q, d6): 0.047742907733029145
BM25(q, d7): 0.3726639461690187
BM25(q, d8): 1.0
--------- Métricas de performance -------
Precisão: 0.5555555555555556
Recall: 0.8333333333333334
MAP: 0.5160714285714286
nDCG: 0.6635732618910778


In [ ]:
precisao_BM25.append(p_BM25_14)
recall_BM25.append(r_BM25_14)
MAP_BM25.append(MAP10_BM25_14)
nDCG_BM25.append(nDCG10_BM25_14)

In [ ]:
ranking_BM25_KG_14, p_KG_14, r_KG_14, MAP10_KG_14, nDCG10_KG_14 = avaliar_ranking_BM25_KG(documentos14, consulta14, relevantes14, ranking_BM25_14)

----------- Sim KG -----------
q = Estrutura política da União Europeia
Sim(q, d0) =  0.7116211469867784
Sim(q, d1) =  0.7344401056687506
Sim(q, d2) =  0.7258486799218041
Sim(q, d3) =  0.6859010668720823
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.7344595869778671
Sim(q, d7) =  0.6968235610976824
Sim(q, d8) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.9662707098102563
Score(q, d1) =  0.624274089818438
Score(q, d2) =  0.7729205496586222
Score(q, d3) =  1.024098755726596
Score(q, d4) =  0.5995099920793582
Score(q, d5) =  0.7472821849574904
Score(q, d6) =  0.6720335566642162
Score(q, d7) =  0.9649639731020487
Score(q, d8) =  1.0
--------- Métricas de performance -------
Precisão: 0.6666666666666666
Recall: 1.0
MAP: 0.5494047619047618
nDCG: 0.6861078732769376


In [ ]:
precisao_BM25_KG.append(p_KG_14)
recall_BM25_KG.append(r_KG_14)
MAP_BM25_KG.append(MAP10_KG_14)
nDCG_BM25_KG.append(nDCG10_KG_14)

### Tema 15

In [ ]:
# ------------Tema 15: Psicologia --------------------
consulta15 = Temas[14]["consulta"]
documentos15 = Temas[14]["documentos"]
relevantes15 = Temas[14]["relevantes"]

In [ ]:
ranking_BM25_15, p_BM25_15, r_BM25_15, MAP10_BM25_15, nDCG10_BM25_15 = avaliar_ranking_BM25(documentos15, consulta15, relevantes15)

----------- Ranking BM25 -----------
q = Os mecanismos cognitivos envolvidos na formação de memória de longo prazo
BM25(q, d0): 0.4094575487237778
BM25(q, d1): 0.33038893908066547
BM25(q, d2): 0.0
BM25(q, d3): 0.0
BM25(q, d4): 0.30208144124021674
BM25(q, d5): 1.0
BM25(q, d6): 0.0
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.6
MAP: 0.6861904761904762
nDCG: 0.7738931964842559


In [ ]:
precisao_BM25.append(p_BM25_15)
recall_BM25.append(r_BM25_15)
MAP_BM25.append(MAP10_BM25_15)
nDCG_BM25.append(nDCG10_BM25_15)

In [ ]:
ranking_BM25_KG_15, p_KG_15, r_KG_15, MAP10_KG_15, nDCG10_KG_15 = avaliar_ranking_BM25_KG(documentos15, consulta15, relevantes15, ranking_BM25_15)

----------- Sim KG -----------
q = Os mecanismos cognitivos envolvidos na formação de memória de longo prazo
Sim(q, d0) =  0.6116211882967622
Sim(q, d1) =  0.6116410176419249
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.9293355587760256
Score(q, d1) =  0.8502838040763017
Score(q, d2) =  0.0
Score(q, d3) =  0.0
Score(q, d4) =  0.30208144124021674
Score(q, d5) =  1.0
Score(q, d6) =  0.0
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.6
MAP: 0.6861904761904762
nDCG: 0.7738931964842559


In [ ]:
precisao_BM25_KG.append(p_KG_15)
recall_BM25_KG.append(r_KG_15)
MAP_BM25_KG.append(MAP10_KG_15)
nDCG_BM25_KG.append(nDCG10_KG_15)

### Tema 16

In [ ]:
# ------------Tema 16: Psicologia --------------------
consulta16 = Temas[15]["consulta"]
documentos16 = Temas[15]["documentos"]
relevantes16 = Temas[15]["relevantes"]

In [ ]:
ranking_BM25_16, p_BM25_16, r_BM25_16, MAP10_BM25_16, nDCG10_BM25_16 = avaliar_ranking_BM25(documentos16, consulta16, relevantes16)

----------- Ranking BM25 -----------
q = As causas da Segunda Guerra Mundial
BM25(q, d0): 0.0
BM25(q, d1): 0.35882043738210584
BM25(q, d2): 0.7825327314929803
BM25(q, d3): 1.0
BM25(q, d4): 1.0
BM25(q, d5): 0.2683252382379662
BM25(q, d6): 0.27471673990061274
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.6220238095238095
nDCG: 0.7397466333385988


In [ ]:
precisao_BM25.append(p_BM25_16)
recall_BM25.append(r_BM25_16)
MAP_BM25.append(MAP10_BM25_16)
nDCG_BM25.append(nDCG10_BM25_16)

In [ ]:
ranking_BM25_KG_16, p_KG_16, r_KG_16, MAP10_KG_16, nDCG10_KG_16 = avaliar_ranking_BM25_KG(documentos16, consulta16, relevantes16, ranking_BM25_16)

----------- Sim KG -----------
q = As causas da Segunda Guerra Mundial
Sim(q, d0) =  0.0
Sim(q, d1) =  0.523476184266295
Sim(q, d2) =  0.43807116944109586
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.5320517991473618
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.0
Score(q, d1) =  0.8037751940084565
Score(q, d2) =  1.1548932255179118
Score(q, d3) =  1.0
Score(q, d4) =  1.0
Score(q, d5) =  0.7205692675132237
Score(q, d6) =  0.27471673990061274
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.7470238095238095
nDCG: 0.8838242945899706


In [ ]:
precisao_BM25_KG.append(p_KG_16)
recall_BM25_KG.append(r_KG_16)
MAP_BM25_KG.append(MAP10_KG_16)
nDCG_BM25_KG.append(nDCG10_KG_16)

### Tema 17

In [ ]:
# ------------Tema 15: Psicologia --------------------
consulta17 = Temas[16]["consulta"]
documentos17 = Temas[16]["documentos"]
relevantes17 = Temas[16]["relevantes"]

In [ ]:
ranking_BM25_17, p_BM25_17, r_BM25_17, MAP10_BM25_17, nDCG10_BM25_17 = avaliar_ranking_BM25(documentos17, consulta17, relevantes17)

----------- Ranking BM25 -----------
q = O funcionamento do DNA nas células
BM25(q, d0): 1.0
BM25(q, d1): 0.28308780533773115
BM25(q, d2): 0.010743507529414
BM25(q, d3): 0.15740091189919275
BM25(q, d4): 0.8777868373611019
BM25(q, d5): 0.0
BM25(q, d6): 0.025014814849504963
--------- Métricas de performance -------
Precisão: 0.7142857142857143
Recall: 1.0
MAP: 0.9266666666666667
nDCG: 0.9747429528567879


In [ ]:
precisao_BM25.append(p_BM25_17)
recall_BM25.append(r_BM25_17)
MAP_BM25.append(MAP10_BM25_17)
nDCG_BM25.append(nDCG10_BM25_17)

In [ ]:
ranking_BM25_KG_17, p_KG_17, r_KG_17, MAP10_KG_17, nDCG10_KG_17 = avaliar_ranking_BM25_KG(documentos17, consulta17, relevantes17, ranking_BM25_17)

----------- Sim KG -----------
q = O funcionamento do DNA nas células
Sim(q, d0) =  0.0
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.6919883424261652
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.0
Score(q, d1) =  0.28308780533773115
Score(q, d2) =  0.010743507529414
Score(q, d3) =  0.15740091189919275
Score(q, d4) =  1.4659769284233422
Score(q, d5) =  0.0
Score(q, d6) =  0.025014814849504963
--------- Métricas de performance -------
Precisão: 0.7142857142857143
Recall: 1.0
MAP: 0.9266666666666667
nDCG: 0.9747429528567879


In [ ]:
precisao_BM25_KG.append(p_KG_17)
recall_BM25_KG.append(r_KG_17)
MAP_BM25_KG.append(MAP10_KG_17)
nDCG_BM25_KG.append(nDCG10_KG_17)

### Tema 18

In [ ]:
# ------------Tema 15: Psicologia --------------------
consulta18 = Temas[17]["consulta"]
documentos18 = Temas[17]["documentos"]
relevantes18 = Temas[17]["relevantes"]

In [ ]:
ranking_BM25_18, p_BM25_18, r_BM25_18, MAP10_BM25_18, nDCG10_BM25_18 = avaliar_ranking_BM25(documentos18, consulta18, relevantes18)

----------- Ranking BM25 -----------
q = As causas da inflação em economias modernas
BM25(q, d0): 1.0
BM25(q, d1): 0.09499564821703056
BM25(q, d2): 0.0
BM25(q, d3): 0.0
BM25(q, d4): 0.0
BM25(q, d5): 0.0
BM25(q, d6): 0.11363514521252364
--------- Métricas de performance -------
Precisão: 0.2857142857142857
Recall: 0.5
MAP: 0.8541666666666666
nDCG: 0.9438661545147249


In [ ]:
precisao_BM25.append(p_BM25_18)
recall_BM25.append(r_BM25_18)
MAP_BM25.append(MAP10_BM25_18)
nDCG_BM25.append(nDCG10_BM25_18)

In [ ]:
ranking_BM25_KG_18, p_KG_18, r_KG_18, MAP10_KG_18, nDCG10_KG_18 = avaliar_ranking_BM25_KG(documentos18, consulta18, relevantes18, ranking_BM25_18)

----------- Sim KG -----------
q = As causas da inflação em economias modernas
Sim(q, d0) =  0.6085623000907422
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
Sim(q, d6) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.5172779550771307
Score(q, d1) =  0.09499564821703056
Score(q, d2) =  0.0
Score(q, d3) =  0.0
Score(q, d4) =  0.0
Score(q, d5) =  0.0
Score(q, d6) =  0.11363514521252364
--------- Métricas de performance -------
Precisão: 0.2857142857142857
Recall: 0.5
MAP: 0.8541666666666666
nDCG: 0.9438661545147249


In [ ]:
precisao_BM25_KG.append(p_KG_18)
recall_BM25_KG.append(r_KG_18)
MAP_BM25_KG.append(MAP10_KG_18)
nDCG_BM25_KG.append(nDCG10_KG_18)

### Tema 19

In [ ]:
# ------------Tema 15: Psicologia --------------------
consulta19 = Temas[18]["consulta"]
documentos19 = Temas[18]["documentos"]
relevantes19 = Temas[18]["relevantes"]

In [ ]:
ranking_BM25_19, p_BM25_19, r_BM25_19, MAP10_BM25_19, nDCG10_BM25_19 = avaliar_ranking_BM25(documentos19, consulta19, relevantes19)

----------- Ranking BM25 -----------
q = A organização da tabela periódica dos elementos
BM25(q, d0): 1.0
BM25(q, d1): 0.0
BM25(q, d2): 0.9786683927735951
BM25(q, d3): 0.8583073847379438
BM25(q, d4): 0.6104463130659766
BM25(q, d5): 0.6104463130659766
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 0.6666666666666666
MAP: 0.6666666666666666
nDCG: 0.8385465274895063


In [ ]:
precisao_BM25.append(p_BM25_19)
recall_BM25.append(r_BM25_19)
MAP_BM25.append(MAP10_BM25_19)
nDCG_BM25.append(nDCG10_BM25_19)

In [ ]:
ranking_BM25_KG_19, p_KG_19, r_KG_19, MAP10_KG_19, nDCG10_KG_19 = avaliar_ranking_BM25_KG(documentos19, consulta19, relevantes19, ranking_BM25_19)

----------- Sim KG -----------
q = A organização da tabela periódica dos elementos
Sim(q, d0) =  0.0
Sim(q, d1) =  0.0
Sim(q, d2) =  0.10272242358018231
Sim(q, d3) =  0.285946519137577
Sim(q, d4) =  0.0
Sim(q, d5) =  0.0
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  1.0
Score(q, d1) =  0.0
Score(q, d2) =  1.06598245281675
Score(q, d3) =  1.1013619260048841
Score(q, d4) =  0.6104463130659766
Score(q, d5) =  0.6104463130659766
--------- Métricas de performance -------
Precisão: 0.3333333333333333
Recall: 0.6666666666666666
MAP: 0.4444444444444444
nDCG: 0.6039071644781281


In [ ]:
precisao_BM25_KG.append(p_KG_19)
recall_BM25_KG.append(r_KG_19)
MAP_BM25_KG.append(MAP10_KG_19)
nDCG_BM25_KG.append(nDCG10_KG_19)

### Tema 20

In [ ]:
# ------------Tema 15: Psicologia --------------------
consulta20 = Temas[19]["consulta"]
documentos20 = Temas[19]["documentos"]
relevantes20 = Temas[19]["relevantes"]

In [ ]:
ranking_BM25_20, p_BM25_20, r_BM25_20, MAP10_BM25_20, nDCG10_BM25_20 = avaliar_ranking_BM25(documentos20, consulta20, relevantes20)

----------- Ranking BM25 -----------
q = Os principais deuses da mitologia grega
BM25(q, d0): 0.0
BM25(q, d1): 0.6848139975284551
BM25(q, d2): 0.6877268481361063
BM25(q, d3): 1.0
BM25(q, d4): 0.07133612128186059
BM25(q, d5): 0.9255940357273252
BM25(q, d6): 0.010934126892314886
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.47619047619047616
nDCG: 0.6325004241086445


In [ ]:
precisao_BM25.append(p_BM25_20)
recall_BM25.append(r_BM25_20)
MAP_BM25.append(MAP10_BM25_20)
nDCG_BM25.append(nDCG10_BM25_20)

In [ ]:
ranking_BM25_KG_20, p_KG_20, r_KG_20, MAP10_KG_20, nDCG10_KG_20 = avaliar_ranking_BM25_KG(documentos20, consulta20, relevantes20, ranking_BM25_20)

----------- Sim KG -----------
q = Os principais deuses da mitologia grega
Sim(q, d0) =  0.0
Sim(q, d1) =  0.0
Sim(q, d2) =  0.0
Sim(q, d3) =  0.0
Sim(q, d4) =  0.6057680104797801
Sim(q, d5) =  0.0
Sim(q, d6) =  0.6057767762105235
----------- Ranking BM25 + Sim KG -----------
Score(q, d0) =  0.0
Score(q, d1) =  0.6848139975284551
Score(q, d2) =  0.6877268481361063
Score(q, d3) =  1.0
Score(q, d4) =  0.5862389301896738
Score(q, d5) =  0.9255940357273252
Score(q, d6) =  0.5258443866712599
--------- Métricas de performance -------
Precisão: 0.42857142857142855
Recall: 0.75
MAP: 0.47619047619047616
nDCG: 0.6325004241086445


In [ ]:
precisao_BM25_KG.append(p_KG_20)
recall_BM25_KG.append(r_KG_20)
MAP_BM25_KG.append(MAP10_KG_20)
nDCG_BM25_KG.append(nDCG10_KG_20)

# Resultados

In [ ]:
precisao_media_BM25 = round(sum(precisao_BM25) / len(precisao_BM25), 3)
recall_media_BM25 = round(sum(recall_BM25) / len(recall_BM25), 3)
MAP_media_BM25 = round(sum(MAP_BM25) / len(MAP_BM25), 3)
nDCG_media_BM25 = round(sum(nDCG_BM25) / len(nDCG_BM25), 3)

print("Média das métricas - BM25")
print("Precisão média: ", precisao_media_BM25)
print("Recall médio: ", recall_media_BM25)
print("MAP médio: ", MAP_media_BM25)
print("nDCG médio: ", nDCG_media_BM25)

Média das métricas - BM25
Precisão média:  0.383
Recall médio:  0.729
MAP médio:  0.746
nDCG médio:  0.849


In [ ]:
precisao_media_BM25_KG = round(sum(precisao_BM25_KG) / len(precisao_BM25_KG), 3)
recall_media_BM25_KG = round(sum(recall_BM25_KG) / len(recall_BM25_KG), 3)
MAP_media_BM25_KG = round(sum(MAP_BM25_KG) / len(MAP_BM25_KG), 3)
nDCG_media_BM25_KG = round(sum(nDCG_BM25_KG) / len(nDCG_BM25_KG), 3)

print("Média das métricas - BM25 + KG")
print("Precisão média: ", precisao_media_BM25_KG)
print("Recall médio: ", recall_media_BM25_KG)
print("MAP médio: ", MAP_media_BM25_KG)
print("nDCG médio: ", nDCG_media_BM25_KG)

Média das métricas - BM25 + KG
Precisão média:  0.433
Recall médio:  0.784
MAP médio:  0.753
nDCG médio:  0.843


In [ ]:
var_p = round((precisao_media_BM25_KG - precisao_media_BM25)/precisao_media_BM25 * 100, 2)
var_r = round((recall_media_BM25_KG - recall_media_BM25)/recall_media_BM25 * 100, 2)
var_map = round((MAP_media_BM25_KG - MAP_media_BM25)/MAP_media_BM25 * 100, 2)
var_ndcg = round((nDCG_media_BM25_KG - nDCG_media_BM25)/nDCG_media_BM25 * 100, 2)

df_metricas = pd.DataFrame({
    'Precisão': [precisao_media_BM25, precisao_media_BM25_KG, f"+{var_p}" if var_p>0 else f"{var_p}"],
    'Recall': [recall_media_BM25, recall_media_BM25_KG, f"+{var_r}" if var_r>0 else f"{var_r}"],
    'MAP': [MAP_media_BM25, MAP_media_BM25_KG, f"+{var_map}"  if var_map>0 else f"{var_map}"],
    'nDCG': [nDCG_media_BM25, nDCG_media_BM25_KG, f"+{var_ndcg}"  if var_ndcg>0 else f"{var_ndcg}"],
},
    index=['BM25', 'BM25 + KG', 'Variação(%)']
)

df_metricas

,Precisão,Recall,MAP,nDCG
BM25,0.383,0.729,0.746,0.849
BM25 + KG,0.433,0.784,0.753,0.843
Variação(%),+13.05,+7.54,+0.94,-0.71
